# 使用telomerecat估计端粒长度
端粒是位于染色体末端的DNA-蛋白质复合体，在人出生时，其长度为8-15kb，每次复制之后，其长度会减少50-200bp。当端粒长度达到临界长度后，会导致复制性衰老和细胞衰亡。因此研究端粒长度对研究人类健康与疾病非常重要。

在这里，我们将为您呈现对2018年发布的计算端粒长度的算法[telomerecat](https://www.nature.com/articles/s41598-017-14403-y)进行复现的过程，并对所得结果进行详细解释。

点击此处可以获得[telomerecat](https://github.com/cancerit/telomerecat)的GitHub链接

# 下载，安装和运行telomerecat以计算端粒长度

在此，我们将介绍如何下载telomereca，如何使用miniconda创建telomereca能够运行的环境，如何调试环境以让telomereca正确运行。

**telomereca是一款以python3.0语言编写的python包，需要在python环境下运行**

**首先，我们使用miniconda创建一个python3.8，命名为telomerecat的python环境**

In [ ]:
# 创建新环境
conda create -n telomerecat python=3.8
# 启动环境
conda  activate telomerecat

以下是作者给出的环境依赖需求：

  * parabam
  * numpy
  * pysam
  * pandas

telomerecat的安装可以使用pip进行，相关代码如下所示：

In [ ]:
# 安装telomerecat
pip install telomerecat

我们也可以使用 ``telomerecat -h`` 查询telomerecat的用法和帮助。

具体来说，telomerecat提供了五种不同的指令，相关功能包含：

* bam2length：估计BAM文件中的端粒长度

* bam2telbam：从BAM文件创建telbams文件

* telbam2length：估计telbams文件中的端粒长度

* csv2length：使用bam2length.CSV文件估计端粒长度

* version：显示版本信息并退出

就我们的科研课题而言，我们只需关注telomerecat bam2length命令。对于这个命令，我们只需要执行以下快速使用代码即可实现从BAM文件中估计端粒长度：``telomerecat bam2length -p8 {BAM 文件路径}``

**-p是您希望 telomerecat bam2length 使用的最大进程数。这应该小于或等于您计算机中的处理器核心数量 [默认值：4]。**



# 运行telomerecat以估计样品的平均端粒长度

你可以使用命令行或西湖大学高性能计算平台的srun系统来运行telomerecat。

下面提供了一个在srun系统中运行telomerecat的模拟代码示例。在这个示例中，我们将计算特定路径下所有的BAM文件的平均端粒长度，并将结果输出到我们手动指定的目标文件output.csv中。

In [ ]:
#!/bin/bash
#SBATCH -p intel-sc3,amd-ep2
#SBATCH -q normal
#SBATCH -J preprocess			
#SBATCH -c 15			
#SBATCH -o /home/douyanmeiLab/hulei/WGS/telomere/Final_output/log/telomerecat.log
#SBATCH --mem=60G
#SBATCH -t 120:00:00

conda activate telomerecat

telomerecat bam2length -p15 \
    --output /home/douyanmeiLab/hulei/WGS/telomere/Final_output/telomercat/output.csv \
    /storage/douyanmeiLab/hulei/Data/Bam/*.bam

# telomerecat结果说明和后处理

以下是来自telomerecat输出的output.csv文件的结果。

对于telomerecat输出的文件，我们只关心Length参数，这个参数即为我们估计的端粒长度。

需要注意的是，在此输出文件中，大部分样本的端粒估计长度都为0。经过查阅日志文件，我们发现这可能意味着我们的原始样本经过预处理以去除端粒reads，或者该样本的平均端粒长度可能非常短。

**在我们的样本中，我们推断大部分样本估计的端粒长度为0的原因是因为样本的测序深度过低。**


![Image title](./img/telomercat_result.png)